# HDA - Project 3
## TASK B1: Activity detection
This task consists of a binary classification, where a gesture denotes activity and thus the model detects wheter there is a gesture label or not (labeled in column 6).

This first cell contains the parameters that can be tuned for code execution:
- subject: select the subject on which to test the model, between [1,4];
- label_col: index of feature column to be selected to perform activity detection, between [0,6];
- folder: directory name where '.mat' files are stored;
- window_size: parameter that sets the length of temporal windows on which to perform the convolution;
- stride: step length to chose the next window.

In [1]:
subject = 1
folder = "./data/full/"
label = 0     # default for task B1
window_size = 15
stride = 5

In [2]:
import utils
import deeplearning
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

Using TensorFlow backend.


Creation of training set and test set

In [3]:
[x_train, y_train, x_test, y_test, n_features, n_classes] = utils.preprocessing(subject,
                                                                                folder,
                                                                                label,
                                                                                window_size,
                                                                                stride,
                                                                                printInfo = True,
                                                                                make_binary = True)

Training samples:  157125 
Test samples:       57536 
Features:             110
Classes: 2

TRAINING SET:
Dataset of Images have shape:  (31422, 15, 110) 
Dataset of Labels have shape:    (31422, 2) 
Fraction of labels:   [0.11036853 0.88963147]

TEST SET:
Dataset of Images have shape:  (11504, 15, 110) 
Dataset of Labels have shape:    (11504, 2) 
Fraction of labels:   [0.1772427 0.8227573]


Preparation of data in a input-suitable form

In [4]:
detection_model = deeplearning.Model1D((window_size,n_features,1), n_classes)
detection_model.summary() # model visualization

ValueError: Input 0 is incompatible with layer conv1d_1: expected ndim=3, found ndim=4

In [ ]:
def f1_loss(y_true, y_pred):
    return 1 - utils.f1_score(np.argmax(y_true), y_pred,average='weighted')

In [ ]:
detection_model.compile(optimizer = Adam(lr=0.01), 
                        loss = "categorical_crossentropy",
                        metrics = ["accuracy"])
                        

input_train = x_train.reshape(x_train.shape[0], window_size, n_features, 1)
input_test = x_test.reshape(x_test.shape[0], window_size, n_features, 1)

checkpointer = ModelCheckpoint(filepath='./data/weights_d.hdf5', verbose=1, save_best_only=True)

detection_model.fit(x = input_train, 
                    y = y_train, 
                    epochs = 5, 
                    batch_size = 128,
                    verbose = 1,
                    validation_data=(input_test, y_test),
                    callbacks=[checkpointer])

In [ ]:
y_pred = detection_model.predict(input_test)
y_pred = np.argmax(y_pred, 1)

print(classification_report(y_test, to_categorical(y_pred)))

In [ ]:
detection_model_best = load_model('./data/weights_d.hdf5')

y_pred = detection_model_best.predict(input_test)
y_pred = np.argmax(y_pred, 1)

print(classification_report(y_test, to_categorical(y_pred)))

## Task B2: gesture recognition
This task consists of a 17-class classification, where gestures are labeled in column 6.

To tune the following parameters, refer to the first cell of task B1:
- subject: select the subject on which to test the model, between [1,4];
- folder: directory name where '.mat' files are stored;
- label_col: index of feature column to be selected to perform activity detection, between [0,6];
- window_size: parameter that sets the length of temporal windows on which to perform the convolution;
- stride: step length to chose the next window.

Here we just need to preserve the different labels, thus we set 'make_binary' to False. We have then 18 different labels, keeping into account the null class, with label 0.

In [ ]:
[x_train, y_train, x_test, y_test, n_features, n_classes] = utils.preprocessing(subject,
                                                                    folder,
                                                                    label,
                                                                    window_size,
                                                                    stride,
                                                                    printInfo = True,
                                                                    make_binary = False,
                                                                    null_class = False)

In [ ]:
n_classes = 4 # OVERWRITE TO BE FIXED

In [ ]:
classification_model = deeplearning.Model1D((window_size,n_features,1), n_classes)
classification_model.summary() # model visualization

In [ ]:
classification_model.compile(optimizer = Adam(lr=0.01), 
                             loss = "categorical_crossentropy", 
                             metrics = ["accuracy"])

input_train = x_train.reshape(x_train.shape[0], window_size, n_features, 1)
input_test = x_test.reshape(x_test.shape[0], window_size, n_features, 1)

checkpointer = ModelCheckpoint(filepath='./data/weights_c.hdf5', verbose=1, save_best_only=True)

classification_model.fit(x = input_train, 
                         y = y_train, 
                         epochs = 10, 
                         batch_size = 128,
                         verbose = 1,
                         validation_data=(input_test, y_test),
                         callbacks=[checkpointer])

In [ ]:
y_pred = classification_model.predict(input_test)
y_pred = np.argmax(y_pred, 1)

print(classification_report(y_test, to_categorical(y_pred)))

In [ ]:
detection_model_best = load_model('./data/weights_c.hdf5')

y_pred = detection_model_best.predict(input_test)
y_pred = np.argmax(y_pred, 1)

print(classification_report(y_test, to_categorical(y_pred)))

## Classification with null class
(detection and classification are performed together)

In [ ]:
[x_train, y_train, x_test, y_test, n_features, n_classes] = utils.preprocessing(subject,
                                                                                folder,
                                                                                label,
                                                                                window_size,
                                                                                stride,
                                                                                printInfo = True,
                                                                                make_binary = False,)

In [ ]:
classification_model = deeplearning.Model1D((window_size,n_features,1), n_classes)
classification_model.summary() # model visualization

In [ ]:
classification_model.compile(optimizer = Adam(lr=0.01), 
                             loss = "categorical_crossentropy", 
                             metrics = ["accuracy"])

input_train = x_train.reshape(x_train.shape[0], window_size, n_features, 1)
input_test = x_test.reshape(x_test.shape[0], window_size, n_features, 1)

checkpointer = ModelCheckpoint(filepath='./data/weights_dc.hdf5', verbose=1, save_best_only=True)

classification_model.fit(x = input_train, 
                         y = y_train, 
                         epochs = 10, 
                         batch_size = 300,
                         verbose = 1,
                         validation_data=(input_test, y_test),
                         callbacks=[checkpointer])

In [ ]:
y_pred = classification_model.predict(input_test)
y_pred = np.argmax(y_pred, 1)

print(classification_report(y_test, to_categorical(y_pred)))

In [ ]:
classification_model_best = load_model('./data/weights_dc.hdf5')

y_pred = classification_model_best.predict(input_test)
y_pred = np.argmax(y_pred, 1)

print(classification_report(y_test, to_categorical(y_pred)))